#### Requirements:

- Torch & Torchvision
- NNI : [open source AutoML toolkit](https://nni.readthedocs.io/en/stable/) > `pip install --upgrade nni`

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/PRUNING

/content/drive/MyDrive/PRUNING


In [5]:
# Import libraries

import nni
import torch

from torch.optim.lr_scheduler import MultiStepLR

from torchvision import datasets, transforms
from torchvision.models import resnet50

#from nni.compression.pytorch import ModelSpeedup
from nni.compression.pytorch.utils.counter import count_flops_params
from nni.compression.pytorch.pruning import LotteryTicketPruner

from copy import deepcopy

In [6]:
# Fixing random number and Device

torch.manual_seed(12345)
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'using {device}')

using cuda


In [7]:
# Loading dataset - in this example CIFAR10

# Dataset located at subfolder 'cifar10'

batch = 128
normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./cifar10', train=True, transform=transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, 4),
        transforms.ToTensor(),
        normalize,
    ]), download=True),
    batch_size=batch, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./cifar10', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=batch, shuffle=False)

Files already downloaded and verified


In [8]:
# Help functions

# Evaluate models
def evaluator(model):
    model.eval()
    correct = 0.0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    acc = 100 * correct / len(test_loader.dataset)
    #print('Accuracy: {}%\n'.format(acc))
    return acc

# Get optimizer, scheduler and criterion
def get_opt_sch_crit(model, _lr=0.1, _momentum=0.9, _weight_decay=5e-4, total_epoch=160):
    optimizer = torch.optim.SGD(model.parameters(), lr=_lr, momentum=_momentum, weight_decay=_weight_decay)
    scheduler = MultiStepLR(optimizer, milestones=[int(total_epoch * 0.5), int(total_epoch * 0.75)], gamma=0.1)
    criterion = torch.nn.CrossEntropyLoss()
    return optimizer, scheduler, criterion

# Default Finetune function (no KD)
def trainer(model, optimizer, criterion):
    model.train()
    for batch, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
          print('Loss: {:.6f}'.format(loss.item()))

In [14]:
from tqdm import tqdm

In [9]:
# Basic functions to distill NN models

# Loss function combining soft and hard labels
def distill_loss(output_student, labels, output_teacher, alpha=1, T=2):
    p = torch.nn.functional.softmax(output_student/T, dim=1)
    q = torch.nn.functional.softmax(output_teacher/T, dim=1)
    z = torch.nn.functional.cross_entropy(output_student, labels)
    l_kl = torch.nn.KLDivLoss(reduction = 'batchmean')(p, q)*(alpha*T*T) + z*(1. - alpha)

    return l_kl

# Loss function with soft labels
# def distill_loss(output, teacher_labels, T=2):
#     p = torch.nn.functional.softmax(output/T, dim=1)
#     q = torch.nn.functional.softmax(teacher_labels/T, dim=1)
#     l_kl = torch.nn.functional.cross_entropy(p, q)
#     return l_kl

# Finetune - Knowledge distilation (KD)
def distiller(student):#, teacher, optimizer):
    optimizer = torch.optim.SGD(student.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
    student.train()
    for batch, (data, target) in enumerate(tqdm(train_loader, total=len(train_loader), desc='KD')):
        data, target = data.to(device), target.to(device)
        output_student = student(data)
        output_teacher = teacher(data)
        loss = distill_loss(output_student, target, output_teacher, alpha=0.9, T=20)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Loss: {loss.item():.6f}')
        #if batch % 100 == 0:
        #    print(f'Distil batch: [{batch+1}|{len(train_loader)} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #        batch_idx * len(data), len(train_loader.dataset),
        #        100. * batch_idx / len(train_loader), loss.item()))

In [10]:
# Starting pipeline...

# random fake input for CIFAR10
fake_input = torch.randn([1, 3, 32, 32]).to(device)

# 1) Loading a high accuracy pretrained model - Resnet50 for CIFAR10

teacher = torch.load('resnet50_cifar10.pth')
acc = evaluator(teacher)
pre_flops, pre_params, _ = count_flops_params(teacher, fake_input, verbose=False)
print(f"Initial parameters: {pre_params/1e6:.2f}M and flops: {pre_flops/1e6:.2f}M, Acc: {acc:.2f}%")

## 2) Or, train the Resnet50 model with CIFAR10

# teacher = resnet50(pretrained=True).to(device) # Teacher model
# epochs = 20
# optimizer, scheduler, criterion = get_opt_sch_crit(teacher, total_epoch=epochs) # train by 20 epochs

Initial parameters: 25.50M and flops: 85.46M, Acc: 77.62%


In [11]:
# # Finetune teacher model from scratch
# pre_best_acc = 0.0
# best_state_dict = None

# for i in range(epochs): # Short train - only 20 epochs
#     trainer(teacher, optimizer, criterion)
#     scheduler.step()
#     acc = evaluator(teacher)
#     if acc > pre_best_acc:
#         pre_best_acc = acc
#         best_state_dict = teacher.state_dict()

# print("Best accuracy: {}".format(pre_best_acc))
# teacher.load_state_dict(best_state_dict)
# torch.save(teacher, 'resnet50_cifar10.pth')
# pre_flops, pre_params, _ = count_flops_params(teacher, fake_input)
# print(f"Initial parameters: {pre_params/1e6:.2f}M and flops: {pre_flops/1e6:.2f}M")

In [12]:
# 2) Starting pruning 

# Copying initial model (pretrained model)
pruned = deepcopy(teacher)
pruned.to(device)
initial_acc = evaluator(pruned)

# checking:
flops, params, _ = count_flops_params(pruned, fake_input, verbose=False)
print(f"Parameters: {pre_params/1e6:.2f}M-{pre_params==params}, Flops: {pre_flops/1e6:.2f}M-{pre_flops==flops}, Acc: {initial_acc:.2f}%")

# Preparing teacher
teacher.to(device)
teacher.eval()

Parameters: 25.50M-True, Flops: 85.46M-True, Acc: 77.62%


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [16]:
# Creating pruned model with Lottery Ticket approach - ref: https://arxiv.org/abs/1803.03635

# Only Conv2d layers will be pruned with ratio = 75% (removing 75% of each layer parameters)
config_list = [{ 'sparsity': 0.75, 'op_types': ['Conv2d'] }]

# The one can apply LotteryTicketPruner with different pruning algorithms: level, l1, l2, fpgm, slim, apoz, mean_activation, taylorfo, admm
# Recommendation: use more iteration as possible

pruner = LotteryTicketPruner(pruned, config_list, pruning_algorithm='fpgm', 
                             total_iteration=5, log_dir='./',
                             finetuner=distiller, speedup=True, 
                             dummy_input=fake_input, evaluator=evaluator, 
                             reset_weight=True)

pruner.compress()
_, model1, masks, _, _ = pruner.get_best_result()

[2022-11-08 12:44:05] WARNING: The old API ...finetuner,speedup,dummy_input,evaluator,reset_weight,pruning_params will be deprecated after NNI v3.0,please using the new one ...evaluator,speedup,reset_weight,pruning_params


[2022-11-08 12:44:06] WARNING: This compressor is not set model and config_list, waiting for reset() or pass this to scheduler.


[2022-11-08 12:44:06] WARNING: The old API ...finetuner,speedup,dummy_input,evaluator,reset_weight will be deprecated after NNI v3.0,please using the new one ...evaluator,speedup,reset_weight


KD: 100%|██████████| 391/391 [00:54<00:00,  7.16it/s]


Loss: -2486.381104
[2022-11-08 12:45:03] simulated prune conv1 remain/total: 49/64


...

INFO:nni.compression.pytorch.speedup.compressor:replace module (name: fc, op_type: Linear)


[2022-11-08 12:50:06] replace linear with new in_features: 1432, out_features: 1000


INFO:nni.compression.pytorch.speedup.compress_modules:replace linear with new in_features: 1432, out_features: 1000


[2022-11-08 12:50:06] speedup done


INFO:nni.compression.pytorch.speedup.compressor:speedup done


In [17]:
flops, params, _ = count_flops_params(model1, fake_input, verbose=False)
print(f'Initial Pretrained model - FLOPs: {pre_flops/1e6:.2f}M, #Params: {pre_params/1e6:.2f}M, Accuracy: {initial_acc: .2f}%')
print(f'Pruned model with LotteryTichet approach based on FPGM policy - FLOPs: {flops/1e6:.2f}M, #Params: {params/1e6:.2f}M, Accuracy: {evaluator(model1)}%')
print(f'Reduction - FLOPs: x{pre_flops/flops:.2f}, #Params: x{pre_params/params:.2f}M, Accuracy: x{initial_acc/evaluator(model1):.2f}')

Initial Pretrained model - FLOPs: 85.46M, #Params: 25.50M, Accuracy:  77.62%
Pruned model with LotteryTichet approach based on FPGM policy - FLOPs: 26.17M, #Params: 8.83M, Accuracy: 22.27%
Reduction - FLOPs: x3.26, #Params: x2.89M, Accuracy: x3.49


In [18]:
# Distilling the pruned model with initial Acc = 22.27%

epochs = 10

model1.to(device)
optimizer, scheduler, criterion = get_opt_sch_crit(model1, _lr=0.01, total_epoch=epochs)

best_acc = 0.0
for i in range(epochs): # short KD
  distiller(model1)
  scheduler.step()
  acc = evaluator(model1)
  if acc > best_acc:
    best_acc = acc
    best_state_dict = model1.state_dict()

  print(f'{i+1}|{epochs} - acc: {acc:.2f}%')

model1.load_state_dict(best_state_dict)

torch.save(model1, 'distilled_{}.pth'.format(round(best_acc,2))) 

flops, params, _ = count_flops_params(model1, fake_input, verbose=False)
print(f'Initial Pretrained model - FLOPs: {pre_flops/1e6:.2f}M, #Params: {pre_params/1e6:.2f}M, Accuracy: {initial_acc: .2f}%')
print(f'Distilled pruned model - FLOPs: {flops/1e6:.2f}M, #Params: {params/1e6:.2f}M, Accuracy: {best_acc:.2f}%')

KD: 100%|██████████| 391/391 [00:49<00:00,  7.95it/s]
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Loss: -2486.247314
1|10 - acc: 69.85%


KD: 100%|██████████| 391/391 [00:48<00:00,  8.01it/s]


Loss: -2486.391357
2|10 - acc: 71.03%


KD: 100%|██████████| 391/391 [00:48<00:00,  8.01it/s]


...

Loss: -2486.387451
8|10 - acc: 73.46%


KD: 100%|██████████| 391/391 [00:48<00:00,  8.09it/s]


Loss: -2486.268311
9|10 - acc: 73.86%


KD: 100%|██████████| 391/391 [00:48<00:00,  8.12it/s]


Loss: -2486.387695
10|10 - acc: 73.85%
Initial Pretrained model - FLOPs: 85.46M, #Params: 25.50M, Accuracy:  77.62%
Distilled pruned model - FLOPs: 26.17M, #Params: 8.83M, Accuracy: 73.86%


> It is important distill for more epochs (maximum as possible) to decrease the accuracy drop between teacher and student! 

- As here the idea is just present the usage example, the values are acceptable once occurred the improvement of the initial accuracy: 22.27% (after pruning) to 73.86% (after KD) with 10 epochs.

In [23]:
# Now one more example using ADMM instead of FPGM


# Copying initial model (pretrained model)
pruned = deepcopy(teacher)
pruned.to(device)
initial_acc = evaluator(pruned)

# checking:
flops, params, _ = count_flops_params(pruned, fake_input, verbose=False)
print(f"Parameters: {pre_params/1e6:.2f}M-{pre_params==params}, Flops: {pre_flops/1e6:.2f}M-{pre_flops==flops}, Acc: {initial_acc:.2f}%")

# Preparing teacher
teacher.to(device)
teacher.eval()


# using the same config list:
config_list = [{ 'sparsity': 0.75, 'op_types': ['Conv2d'] }]

# this algorithm requires a traced optimizer:
traced_optimizer = nni.trace(torch.optim.SGD)(pruned.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# pruning_params dic contains specific parameters for pruning algorithm, in this case additional parameters for ADMM approach
pruner = LotteryTicketPruner(pruned, config_list, pruning_algorithm='admm', 
                             total_iteration=5, log_dir='./',
                             finetuner=distiller, speedup=True, 
                             dummy_input=fake_input, evaluator=evaluator, 
                             reset_weight=True,
                             pruning_params={'trainer':trainer,'traced_optimizer':traced_optimizer,'criterion':criterion,'iterations':3, 'training_epochs':1, 'granularity':'coarse-grained'})

pruner.compress()
_, model2, masks, _, _ = pruner.get_best_result()

Parameters: 25.50M-True, Flops: 85.46M-True, Acc: 77.62%
[2022-11-08 13:21:46] WARNING: The old API ...finetuner,speedup,dummy_input,evaluator,reset_weight,pruning_params will be deprecated after NNI v3.0,please using the new one ...evaluator,speedup,reset_weight,pruning_params


[2022-11-08 13:21:47] WARNING: The old API trainer,traced_optimizer,criterion,iterations,training_epochs,granularity will be deprecated after NNI v3.0, please using the new one evaluator,iterations,training_epochs,granularity


[2022-11-08 13:21:47] WARNING: This compressor is not set model and config_list, waiting for reset() or pass this to scheduler.


[2022-11-08 13:21:47] WARNING: The old API ...finetuner,speedup,dummy_input,evaluator,reset_weight will be deprecated after NNI v3.0,please using the new one ...evaluator,speedup,reset_weight


KD: 100%|██████████| 391/391 [00:56<00:00,  6.95it/s]

...

INFO:nni.compression.pytorch.speedup.compressor:replace module (name: fc, op_type: Linear)


[2022-11-08 13:37:31] replace linear with new in_features: 1437, out_features: 1000


INFO:nni.compression.pytorch.speedup.compress_modules:replace linear with new in_features: 1437, out_features: 1000


[2022-11-08 13:37:31] speedup done


INFO:nni.compression.pytorch.speedup.compressor:speedup done


In [24]:
flops, params, _ = count_flops_params(model2, fake_input, verbose=False)
print(f'Initial Pretrained model - FLOPs: {pre_flops/1e6:.2f}M, #Params: {pre_params/1e6:.2f}M, Accuracy: {initial_acc: .2f}%')
print(f'Pruned model with LotteryTichet approach based on ADMM method - FLOPs: {flops/1e6:.2f}M, #Params: {params/1e6:.2f}M, Accuracy: {evaluator(model2)}%')
print(f'Reduction - FLOPs: x{pre_flops/flops:.2f}, #Params: x{pre_params/params:.2f}M, Accuracy: x{initial_acc/evaluator(model2):.2f}')

Initial Pretrained model - FLOPs: 85.46M, #Params: 25.50M, Accuracy:  77.62%
Pruned model with LotteryTichet approach based on ADMM method - FLOPs: 26.30M, #Params: 8.87M, Accuracy: 21.89%
Reduction - FLOPs: x3.25, #Params: x2.88M, Accuracy: x3.55


In [25]:
# Distilling the pruned model with initial Acc = 21.89%

epochs = 10

model2.to(device)
optimizer, scheduler, criterion = get_opt_sch_crit(model2, _lr=0.01, total_epoch=epochs)

best_acc = 0.0
for i in range(epochs): # short KD
  distiller(model2)
  scheduler.step()
  acc = evaluator(model2)
  if acc > best_acc:
    best_acc = acc
    best_state_dict = model2.state_dict()

  print(f'{i+1}|{epochs} - acc: {acc:.2f}%')

model2.load_state_dict(best_state_dict)

torch.save(model2, 'distilled_admm_{}.pth'.format(round(best_acc,2))) 

flops, params, _ = count_flops_params(model2, fake_input, verbose=False)
print(f'Initial Pretrained model - FLOPs: {pre_flops/1e6:.2f}M, #Params: {pre_params/1e6:.2f}M, Accuracy: {initial_acc: .2f}%')
print(f'Distilled pruned model - FLOPs: {flops/1e6:.2f}M, #Params: {params/1e6:.2f}M, Accuracy: {best_acc:.2f}%')

KD: 100%|██████████| 391/391 [00:50<00:00,  7.76it/s]
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Loss: -2486.383545
1|10 - acc: 69.44%


KD: 100%|██████████| 391/391 [00:49<00:00,  7.92it/s]


Loss: -2486.386719
2|10 - acc: 70.83%


KD: 100%|██████████| 391/391 [00:49<00:00,  7.92it/s]


...

Loss: -2486.389893
8|10 - acc: 73.23%


KD: 100%|██████████| 391/391 [00:49<00:00,  7.93it/s]


Loss: -2486.411133
9|10 - acc: 73.47%


KD: 100%|██████████| 391/391 [00:49<00:00,  7.91it/s]


Loss: -2486.378174
10|10 - acc: 73.67%
Initial Pretrained model - FLOPs: 85.46M, #Params: 25.50M, Accuracy:  77.62%
Distilled pruned model - FLOPs: 26.30M, #Params: 8.87M, Accuracy: 73.67%


**Considerations:**

> Both examples using Lottery Ticket approach with FPGM and ADMM methods demonstrated the powerfull of such compression algorithm to find a minimal representative neural network.

> The usage of Knowledge distillation provides a useful way to recovery the accuracy of pruned models in few epochs.

> Others methods should be evaluated for determining the best way according to aim required by the project.

> The usage of high accuracy teacher model improve the efficiency of pruned model - considering start with the best one according to the target dataset.
